In [1]:
import pandas as pd
import re
import datetime
import numpy as np

# 2009 Data Preprocess

In [22]:
pbp_data_09 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2009.csv")
pbp_data_10 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2010.csv")
pbp_data_09 = pd.concat([pbp_data_09,pbp_data_10,pbp_data_10_post,pbp_data_10_pre])

/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_92231/488309357.py:1: DtypeWarning: Columns (42,167,168,175,176,179,180,183,184,189,190,191,192,204,205,206,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_09 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2009.csv")
/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_92231/488309357.py:2: DtypeWarning: Columns (42,167,168,169,170,175,176,179,180,183,184,189,190,191,192,195,196,204,205,206,219,220,221,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_10 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2010.csv")


In [23]:
highlights09 = pd.read_csv("./NFLGameData - 2009.csv")

In [24]:
relevant_cols = pbp_data_09.drop(columns=(list(pbp_data_09.iloc[:,58:110].columns) + ["desc","home_timeouts_remaining","home_timeouts_remaining","timeout","timeout_team","play_id","game_id","game_date"])).columns

In [25]:
highlights09["intervals"] = highlights09.apply(lambda x: pd.Interval(x["Play Time End"],x["Play Time Start"],closed="both"),axis=1)
highlights09["Date"] = pd.to_datetime(highlights09["Date"])
pbp_data_09["game_date"] = pd.to_datetime(pbp_data_09["game_date"])

In [26]:
merged09 = pbp_data_09.merge(highlights09,left_on=["home_team","away_team"], right_on=["Home","Away"])

In [27]:
merged09[["home_team","away_team","game_date","Transcript"]].drop_duplicates()

,home_team,away_team,game_date,Transcript
0,MIN,SF,2009-09-27,Everybody out. Favre rolls. Watch the throw de...
1,MIN,SF,2009-09-27,And it's blocked! And it's picked up by the 49...
2,MIN,SF,2009-09-27,Percy Harvin brings it back from the goal line...
549,CHI,GB,2009-12-13,It's first and goal from inside the one. Hando...
715,CHI,GB,2010-09-27,It's first and goal from inside the one. Hando...
874,CHI,GB,2011-01-23,It's first and goal from inside the one. Hando...
1053,PIT,GB,2009-12-20,Here is one aired out for Mike Wallace. What a...
1054,PIT,GB,2009-12-20,"Down the middle for Jennings, what a perfect t..."
1439,ARI,GB,2010-01-03,"On first and goal, Hightower slams it in for t..."
1587,NO,MIN,2010-09-09,Peterson again! How about Adrian Peterson and ...


In [28]:
highlights09

,Home,Away,Date,URL,Transcript,Play Time Start,Play Time End,intervals
0,PIT,GB,2009-12-20,https://www.youtube.com/watch?v=5mgHYywK1jM&li...,Here is one aired out for Mike Wallace. What a...,3568,3558,"[3558, 3568]"
1,PIT,GB,2009-12-20,https://www.youtube.com/watch?v=5mgHYywK1jM&li...,"Down the middle for Jennings, what a perfect t...",3248,3232,"[3232, 3248]"
2,ARI,GB,2010-01-10,https://www.youtube.com/watch?v=IMhGuBvyp5E&li...,"On first and goal, Hightower slams it in for t...",3368,3364,"[3364, 3368]"
3,NO,MIN,2010-01-24,https://www.youtube.com/watch?v=-rUp-t386sU&li...,Peterson again! How about Adrian Peterson and ...,3282,3275,"[3275, 3282]"
4,MIN,SF,2009-09-27,https://www.youtube.com/watch?v=fKcYHlWhO9w&li...,Everybody out. Favre rolls. Watch the throw de...,12,10,"[10, 12]"
5,MIN,SF,2009-09-27,https://www.youtube.com/watch?v=fKcYHlWhO9w&li...,And it's blocked! And it's picked up by the 49...,1806,1800,"[1800, 1806]"
6,MIN,SF,2009-09-27,https://www.youtube.com/watch?v=fKcYHlWhO9w&li...,Percy Harvin brings it back from the goal line...,1187,1173,"[1173, 1187]"
7,CHI,GB,2010-09-27,https://www.youtube.com/watch?v=PnhyDkQiD6w&li...,It's first and goal from inside the one. Hando...,726,722,"[722, 726]"
8,NYG,GB,2010-12-26,https://www.youtube.com/watch?v=YORmPyLcRZM&li...,"From the 15 down by 7, Eli airs it out for Man...",2125,2113,"[2113, 2125]"


In [29]:
def in_interval(row):
    return row["game_seconds_remaining"] in row["intervals"]

In [30]:
plays09 = merged09[merged09.apply(lambda x: x["game_seconds_remaining"] in x["intervals"],axis=1)]

In [31]:
touches = plays09[plays09["touchdown"] == 1].groupby("intervals")['touchdown'].sum()

In [32]:
mi_index = touches[touches == 1].index

In [33]:
plays09 = plays09[(plays09["touchdown"] == 1) & (plays09["intervals"].isin(mi_index))]

In [34]:
X = plays09[relevant_cols.to_list()+["Transcript"]]

In [35]:
ret_list = []
for i in range(X.shape[0]):
    ret_list += [{"json":X[relevant_cols].iloc[i].replace(0,np.nan).dropna().to_json(),
                  "transcript":X["Transcript"].iloc[i]}]

In [36]:
highlights09[~highlights09["Transcript"].isin(X.Transcript)]

,Home,Away,Date,URL,Transcript,Play Time Start,Play Time End,intervals
1,PIT,GB,2009-12-20,https://www.youtube.com/watch?v=5mgHYywK1jM&li...,"Down the middle for Jennings, what a perfect t...",3248,3232,"[3232, 3248]"
2,ARI,GB,2010-01-10,https://www.youtube.com/watch?v=IMhGuBvyp5E&li...,"On first and goal, Hightower slams it in for t...",3368,3364,"[3364, 3368]"
3,NO,MIN,2010-01-24,https://www.youtube.com/watch?v=-rUp-t386sU&li...,Peterson again! How about Adrian Peterson and ...,3282,3275,"[3275, 3282]"
8,NYG,GB,2010-12-26,https://www.youtube.com/watch?v=YORmPyLcRZM&li...,"From the 15 down by 7, Eli airs it out for Man...",2125,2113,"[2113, 2125]"


In [37]:
merged09["Transcript"].unique()

array(['Everybody out. Favre rolls. Watch the throw deep it goes. To the end zone for Greg Lewis. Touchdown! Touchdown! Favre did it!',
       "And it's blocked! And it's picked up by the 49ers! Nate Clements goes in for a touchdown! What a play! I believe Ray McDonald blocked it, Nate Clements picked it up and the 49ers have a chance to take the lead!",
       "Percy Harvin brings it back from the goal line. He's got some room watch out! Percy's going! Percy Harvin is gone! Touchdown Vikings!",
       "It's first and goal from inside the one. Handoff. Touchdown Chester Taylor!",
       'Here is one aired out for Mike Wallace. What a start. Touch down Pittsburgh! ',
       'Down the middle for Jennings, what a perfect throw. Jennings still going. Inside the 20. And the Packers have an answer. 83 yards for Greg Jennings, his 4th touchdown catch of the season.',
       'On first and goal, Hightower slams it in for the touchdown. ',
       'Peterson again! How about Adrian Peterson and th

In [40]:
ret_list

[{'json': '{"home_team":"MIN","away_team":"SF","posteam":"MIN","posteam_type":"home","defteam":"SF","side_of_field":"SF","yardline_100":26.0,"quarter_seconds_remaining":6.0,"half_seconds_remaining":6.0,"game_seconds_remaining":1806.0,"game_half":"Half1","drive":14,"sp":1,"qtr":2,"down":2.0,"time":"00:06","yrdln":"SF 26","ydstogo":20,"ydsnet":54,"play_type":"field_goal","field_goal_result":"blocked","kick_distance":44.0,"away_timeouts_remaining":1,"td_team":"SF","posteam_timeouts_remaining":1.0,"defteam_timeouts_remaining":1.0,"total_home_score":13,"total_away_score":13,"posteam_score":13.0,"defteam_score":7.0,"score_differential":6.0,"posteam_score_post":13.0,"defteam_score_post":13.0,"touchdown":1.0,"field_goal_attempt":1.0,"kicker_player_name":"R.Longwell","kicker_player_id":"00-0010072","blocked_player_id":"00-0025484","blocked_player_name":"R.McDonald"}',
  'transcript': "And it's blocked! And it's picked up by the 49ers! Nate Clements goes in for a touchdown! What a play! I believ